# DataLoader
- 학습을 하기 위해서는 데이터를 불러와야 한다.
- 이전에는 데이터를 하나하나 가져오는 방식으로 불러옴
- Pytorch에 내장된 함수를 이용해서 편리하게 불러오는 방법을 알아보자

### Custom Datasets
- 기존에는 datasets을 바로 다운받아서 사용했다.(MNIST나 CIFAR10의 경우)
- torch에 있는 Dataset를 직접 만들어서 DataLoader로 불러오기
- Dataset 함수의 형태를 이해하고 customize 해보기

In [3]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

# Datasets 만들기
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Original Method
- 필요한 데이터셋을 다운로드한 후 불러오기
- 불러 온 데이터셋을 바로 Dataloader에 입력

In [8]:
import torchvision

trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=False, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=1)

print(trainset)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


# Custom Dataset
Dataset을 불러오는 class를 직접 코딩
1. 데이터의 위치 파악
2. 데이터와 해당 데이터의 레이블 확인

In [15]:
from torch.utils.data import Dataset

class MyDataset(Dataset): 
    def __init__(self):
        pass
    def __len__(self):
        pass
    def __getitem__(self, idx): 
        pass

In [39]:
from torch.utils.data import Dataset

class MyDataset(Dataset): 
    def __init__(self):
        self.x = [[100,80,100,70],
                  [70, 95, 92, 67],
                  [60, 88, 73, 92],
                  [80, 62, 91, 79]]
        self.y = [[150],[200],[200],[300]]
        
    def __len__(self):
        data_len = len(self.x)
        
        return data_len
        
    def __getitem__(self, idx):
        
        x = torch.Tensor(self.x)[idx]
        y = torch.Tensor(self.y)[idx]
        
        return x, y

mydata = MyDataset()
print(mydata)

In [41]:
from torch.utils.data import DataLoader

data = DataLoader(mydata, batch_size=1)
iter(data).next()

[tensor([[100.,  80., 100.,  70.]]), tensor([[150.]])]

# 이미지 데이터 정의

In [233]:
import glob
import torchvision.transforms as transforms
import re

from torch.utils.data import Dataset
from PIL import Image


class MyImageData(Dataset):
    
    def __init__(self, data_path, class_to_label, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.class_to_label = class_to_label
        self.image_list = glob.glob(self.data_path + '/*.jpg')
        
    def __len__(self):
        
        return len(self.image_list)
    
    def __getitem__(self, idx):
        
        file_name = self.image_list[idx]
        
        img = Image.open(file_name)

        class_name = re.findall('[a-zA-Z]+', file_name)[-2]
        label = self.class_to_label[class_name]
        label = torch.tensor(label)
        
        img = self.transform(img)

        return img, label
    
    
data_path = 'data/image'
class_to_label = {'dog' : 1, 'cat' : 2}
transform = transforms.ToTensor()

mydataset = MyImageData(data_path, class_to_label, transform=transform)

data = DataLoader(mydataset)

print(iter(data).next())

[tensor([[[[0.5843, 0.5843, 0.5843,  ..., 0.5882, 0.5882, 0.5882],
          [0.5843, 0.5843, 0.5843,  ..., 0.5922, 0.5922, 0.5922],
          [0.5882, 0.5882, 0.5882,  ..., 0.5922, 0.5922, 0.5922],
          ...,
          [0.1686, 0.1686, 0.1843,  ..., 0.0784, 0.0863, 0.0902],
          [0.1608, 0.1490, 0.1529,  ..., 0.0902, 0.0980, 0.1020],
          [0.1529, 0.1294, 0.1255,  ..., 0.1059, 0.1137, 0.1137]],

         [[0.6941, 0.6941, 0.6941,  ..., 0.6980, 0.6980, 0.6980],
          [0.6941, 0.6941, 0.6941,  ..., 0.7020, 0.7020, 0.7020],
          [0.6980, 0.6980, 0.6980,  ..., 0.7020, 0.7020, 0.7020],
          ...,
          [0.2549, 0.2510, 0.2706,  ..., 0.1451, 0.1529, 0.1569],
          [0.2471, 0.2314, 0.2431,  ..., 0.1569, 0.1647, 0.1686],
          [0.2392, 0.2157, 0.2157,  ..., 0.1725, 0.1804, 0.1804]],

         [[0.7882, 0.7882, 0.7882,  ..., 0.7843, 0.7843, 0.7843],
          [0.7882, 0.7882, 0.7882,  ..., 0.7882, 0.7882, 0.7882],
          [0.7922, 0.7922, 0.7922,  ..., 

# ImageFolder
- Torchvision 라이브러리에 있는 함수
- 데이터들이 각각의 레이블파일로 구축되어있을 때 사용하는 함수
> **ImageFolder는 데이터를 3채널로만 출력을 하기 때문에 1채널만 있는 이미지에 대해서는 따로 수정을 해줘야한다.**

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

transform = transforms.ToTensor()

datasets = ImageFolder(root='../../data/fruit', transform=transform)
print(datasets)
# Dataset ImageFolder
#     Number of datapoints: 8513
#     Root location: ../../data/fruit

data = DataLoader(datasets, batch_size=1)
>>> print(iter(data).next())